In [22]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import yfinance as yf
import pandas as pd
import datetime as dt
import requests
from bs4 import BeautifulSoup
import nltk
from tqdm import tqdm

from utils import *

## News extraction

In [24]:
ticker_news = yf.Ticker("AAPL").news

dict_list = []

labels = {0:'neutral', 1:'positive', 2:'negative'}

for news in tqdm(ticker_news):
    timestamp = news['providerPublishTime']
    datetime = dt.datetime.fromtimestamp(timestamp)

    url = news['link']
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    article_elements = soup.find_all(attrs={"role": "article"})[0]

    sentences = nltk.sent_tokenize(article_elements.get_text())

    news_dict = {
        'date': datetime,
        'title': news['title'],
        'link': news['link'],
    }

    news_dict.update(get_sentiment(sentences))

    dict_list.append(news_dict)

    print(news_dict)

df = pd.DataFrame(dict_list, index=[x['date'] for x in dict_list])

display(df)

  0%|          | 0/8 [00:00<?, ?it/s]